In [ ]:
from phoenix.otel import register

register(
	project_name="dspy-demo",
	endpoint="http://localhost:6006/v1/traces",
	verbose=False,
    auto_instrument=True,
)

In [ ]:
import dspy

openai = dspy.LM("openai/gpt-4.1-nano", cache=False)
dspy.configure(lm=openai)

# Signatures

Signatures in DSPy serve the same purpose as a function signature in Python. They define the inputs and outputs of a DSPy module, allowing DSPy to understand what to expect and what to produce. Signatures are used to validate the inputs and outputs of a module, ensuring that they conform to the expected types and formats.

In [ ]:
simple_signature = dspy.Signature("question -> answer")
predictor = dspy.Predict(simple_signature)

predictor(question="When did Iowa become a state?")

In [ ]:
typed_signature = dspy.Signature("question: str -> answer: str")
typed_predictor = dspy.Predict(typed_signature)

typed_predictor(question="When were fireworks invented?")

In [ ]:
from typing import Optional

class ClassSignature(dspy.Signature):
	"""
	Answers a question based on the provided input.
	"""

	question: str = dspy.InputField()
	contexts: Optional[list[str]] = dspy.InputField(desc="List of contexts to answer the question from.")
	answer: str = dspy.OutputField(desc="Answer to the question, which is succinct and provides no additional information or preamble.")

class_predictor = dspy.Predict(ClassSignature)

class_predictor(question="What is DSPy?", contexts=None)

In [ ]:
class_predictor(question="What is DSPy?", contexts=[
    """DSPy is a declarative framework for building modular AI software. It allows you to iterate fast on structured code, rather than brittle strings, and offers algorithms that compile AI programs into effective prompts and weights for your language models, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.

	Instead of wrangling prompts or training jobs, DSPy (Declarative Self-improving Python) enables you to build AI software from natural-language modules and to generically compose them with different models, inference strategies, or learning algorithms. This makes AI software more reliable, maintainable, and portable across models and strategies."""
])

# Modules

Modules are used in DSPy to define structure to an AI program. They are the building blocks of DSPy programming and there are several built-in modules that can be leveraged.

* dspy.Predict: Standard zero-shot prediction module.
* dspy.ChainOfThought: Chain of thought module that allows for reasoning over multiple steps.
* dspy.ReAct: ReAct module that allows for reasoning and action-taking (tool calling).
* dspy.ProgramOfThought: A DSPy module that runs Python programs to solve a problem.
* More modules can be found in the [DSPy documentation](https://dspy.ai/api/modules/Module/).

In [ ]:
chain_of_thought = dspy.ChainOfThought(typed_signature)

print("Basic prediction:", predictor(question="How many r's are in the word strawberry?").answer)

chain_of_thought(question="How many r's are in the word strawberry?")

In [ ]:
from typing import Literal

@dspy.Tool
def do_math(x: float, y: float, operation: Literal['+', '-', '*', '/', '^']) -> float:
	"""
	Performs a mathematical operation on two numbers.
	"""
	if operation == '+':
		return x + y
	elif operation == '-':
		return x - y
	elif operation == '*':
		return x * y
	elif operation == '/':
		return x / y
	elif operation == '^':
		return x ** y
	else:
		raise ValueError("Invalid operation. Supported operations are: '+', '-', '*', '/', '^'.")

math_signature = dspy.Signature("question -> answer: float")

math_agent = dspy.ReAct(
	math_signature,
	tools=[do_math],
)

In [ ]:
from pprint import pprint
math_response = math_agent(question="Raise the largest prime number less than 100 to the power of 2.")

print("Trajectory")
pprint(math_response.trajectory)
print("reasoning:", math_response.reasoning)
print("answer:", math_response.answer)

# MCP

Model context protocol is everywhere in 2025 and DSPy natively supports it! Let's see a demo of an MCP server that allows an AI to control my home.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import os
from dotenv import load_dotenv
import dspy

load_dotenv()

openai = dspy.LM("openai/gpt-4.1-mini", cache=False)
dspy.configure(lm=openai)


server_params = StdioServerParameters(
    command="docker",
    args=["run",
          "-i",
          "--rm",
          "-e",
          "HA_URL",
          "-e",
          "HA_TOKEN",
          "voska/hass-mcp"],
    env={
		"HA_URL": os.environ["HA_URL"],
		"HA_TOKEN": os.environ["HA_TOKEN"],
    },
)

class HomeAssistantSignature(dspy.Signature):
    """
    Useful entities in the Home Assistant system:
    Family room lamps = switch.lamps
    Thermostat = climate.family_room

    TrueNAS / NAS is a custom integration. main_pool is the hard drive pool and speedy_boy is the ssd pool for VMs 
    
    Temperatures should be in Fahrenheit unless they are computer temperatures.
    """

    question: str = dspy.InputField()
    answer: str = dspy.OutputField(desc="A succinct answer to the question or the result of the action taken.")


async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()

        dspy_tools = [dspy.Tool.from_mcp_tool(session, tool) for tool in tools.tools]

        agent = dspy.ReAct(HomeAssistantSignature, tools=dspy_tools,)

        history = dspy.History(messages=[])

        while (question := input("You: ")) != "exit":

            response = await agent.acall(question=question, history=history)

            print("Agent:", response.answer, flush=True)

            history.messages.append({"question": question, "answer": response.answer})